**Q4**

In [46]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nba_df=nba_df[nba_df.year == 2018]
    nba_df['team'] = nba_df.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if '*' in team else team)
    #print(nba_df)
    nba_df['team'] = nba_df.team.apply(lambda team: re.findall("([\w\s]*)\(", team)[0] if '(' in team else team)
    #print(nba_df)

    cities['NBA'] = cities.NBA.apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if '[' in team else team)
    #print(cities['NBA'] )
    nba_df = nba_df[["team", "W", "L"]]
    #print(nba_df)
    nba_df['team_only'] = nba_df.team.apply(lambda x:x.rsplit(None,1)[-1])

    nba_df["team_only"] = nba_df["team_only"].replace({"Blazers": "Trail Blazers",
                                        "Knicks": "KnicksNets",
                                        "Nets": "KnicksNets",
                                        "Lakers": "LakersClippers",
                                        "Clippers": "LakersClippers"}) 
   
    #nba_df.to_excel("team-only.xlsx")

    cities.rename(columns={"Population (2016 est.)[8]":"population", "NBA":"team_only"},inplace=True )
    #cities.to_excel("Metropolitan.xlsx")
    nba_df=nba_df[~nba_df.W.str.contains("Division")]
    #print(nba_df)
    #nba_df=nba_df[["team_only","W","L"]]
    nba_df.W=nba_df.W.astype("float")
    nba_df.L=nba_df.L.astype("float")
    nba_df=nba_df.groupby(by="team_only")[["W","L"]].mean()
    #print(nba_df)
    nba_df.reset_index(inplace=True)
    #print(nba_df)
    merged_df=pd.merge(nba_df,cities, how="inner",on="team_only")
    merged_df.W=nba_df.W.astype("float")
    merged_df.L=nba_df.L.astype("float")
    merged_df['win_loss_ratio']=merged_df.W/(merged_df.W + merged_df.L)
    merged_df.to_excel("blabla.xlsx")

    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merged_df["population"],errors='coerce')# pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df["win_loss_ratio"],errors='coerce') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.17657160252844611

0.01230899645574425

**Q3 MLB**

In [140]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    mlb_df=mlb_df[mlb_df.year == 2018]
    mlb_df = mlb_df[["team", "W", "L"]]
    #print(mlb_df.shape) #(30,3) 
    '''mlb_df['team'] = mlb_df.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if '*' in team else team)
    #print(mlb_df.shape) #(30,11)

    mlb_df['team'] = mlb_df.team.apply(lambda team: re.findall("([\w\s]*)\(", team)[0] if '(' in team else team)
    #print(mlb_df.shape) #(30,11)'''

    cities['MLB'] = cities.MLB.apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if '[' in team else team)
    #print(cities['MLB'].shape) # (51,)
    mlb_df["team"]=mlb_df['team'].replace({"Boston Red Sox":"Boston RedSox",
                                           "Toronto Blue Jays":"Toronto BlueJays",
                                           "Los Angeles Dodgers": "DodgersAngels",
                                            "Los Angeles Angels" : "DodgersAngels",
                                            "New York Yankees":"YankeesMets",   
                                            "New York Mets":"YankeesMets",
                                            "San Francisco Giants":"GiantsAthletics",
                                            "Oakland Athletics":"GiantsAthletics",
                                            "Chicago Cubs":"CubsWhiteSox",
                                            "Chicago White Sox":"CubsWhiteSox"})
    mlb_df['team_only'] = mlb_df.team.apply(lambda x:x.rsplit(None,1)[-1])
    #print(mlb_df.shape) #(30.4)
    mlb_df.to_excel("mlb_team-only.xlsx")

    cities.rename(columns={"Population (2016 est.)[8]":"population", "MLB":"team_only"},inplace=True)
    cities["team_only"]=cities['team_only'].replace({"Red Sox":"RedSox",
                                                    "Blue Jays":"BlueJays",
                                                    "CubsWhite Sox":"CubsWhiteSox"})
    #print(cities.shape) #(51,6)
    cities.to_excel("Metropolitan.xlsx")
    #mlb_df=mlb_df[~mlb_df.W.str.contains("Division")]
    #print(mlb_df)
    mlb_df=mlb_df[["team_only","W","L"]]
    mlb_df.W=mlb_df.W.astype("float")
    mlb_df.L=mlb_df.L.astype("float")
    mlb_df=mlb_df.groupby(by="team_only")[["W","L"]].mean()
    #print(mlb_df.shape) #(26,2) + the index column

    mlb_df.reset_index(inplace=True)
    #print(mlb_df)
    merged_df=pd.merge(mlb_df,cities, how="inner",on="team_only")
    merged_df.to_excel("mlb_merged_teamonly.xlsx")

    print("according to Visual Studio Environment (26,8)shape: ",merged_df.shape) #(26,8) according to Visual Studio Environment

    merged_df.W=mlb_df.W.astype("float")
    merged_df.L=mlb_df.L.astype("float")

    merged_df['win_loss_ratio']=merged_df.W/(merged_df.W + merged_df.L)
    #print(merged_df['win_loss_ratio'])

    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = pd.to_numeric(merged_df["population"],errors='coerce')# pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df["win_loss_ratio"],errors='coerce') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q2: There should be 28 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

according to Visual Studio Environment (26,8)shape:  (26, 8)


0.15052304487104848